# NA Filling

In [67]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import scipy.stats as stats
from scipy.stats import chi2_contingency, mode
import numpy as np
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import f_oneway

import math

In [68]:
file_path = 'Airbnb_Data.csv'
original_df = pd.read_csv(file_path)

In [69]:
# Which columns are NA / Null / INF / None/ 0

df = original_df.copy()

missing_values = ['NA', 'NaN', 'nan', 'Null', 'INF', '-INF', 'None', 0, '']

# Initialize a dictionary to store columns with missing or invalid values
columns_with_issues = {}

# Check for missing or invalid values in each column
for column in df.columns:
    # Check for NaN, Null, and None values
    if df[column].isna().any() or df[column].isnull().any() or df[column].eq(None).any():
        columns_with_issues[column] = 'Missing or Null'
    # Check for INF and -INF values
    elif (df[column] == np.inf).any() or (df[column] == -np.inf).any():
        columns_with_issues[column] = 'INF or -INF'
    # Check for specific representations (e.g., 'NA', 'Null', 'None', 0)
    elif df[column].astype(str).isin(missing_values).any():
        columns_with_issues[column] = 'Specific Representation'

# Print columns with missing or invalid values
print("Columns with missing or invalid values:")
for column, issue in columns_with_issues.items():
    print(f"{column}: {issue}")

Columns with missing or invalid values:
bathrooms: Missing or Null
first_review: Missing or Null
host_has_profile_pic: Missing or Null
host_identity_verified: Missing or Null
host_response_rate: Missing or Null
host_since: Missing or Null
last_review: Missing or Null
neighbourhood: Missing or Null
review_scores_rating: Missing or Null
thumbnail_url: Missing or Null
zipcode: Missing or Null
bedrooms: Missing or Null
beds: Missing or Null


## Per Column Analysis
- Is it MCAR?
- Is it MAR? missing because of other variables
- IS it MNAR? missing because of that column


- MCAR/MAR -> If categorical, use logistic reg/multiple imput. If continuous, use linear reg/multiple imput. Assuming they are better than using mean/median/mode
Can consider KNN/Random Forest


Ignore thumbnail_url


- chi-square

In [70]:
# convert to 

In [71]:
IGNORE_COLUMNS = ["id", "log_price", "amenities", "name", "thumbnail_url", "description", "zipcode"]
COLUMNS_TO_CHECK_CORRELATION = list(original_df.columns)

CATEGORICAL_COLUMNS = ['property_type', 'room_type', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
DATE_COLUMNS = ['first_review', 'host_since', 'last_review']
NUMERICAL_COLUMNS = ['accommodates', 'bathrooms', 'host_response_rate', 'number_of_reviews', 'review_scores_rating', 'bedrooms', 'beds']


for col in IGNORE_COLUMNS:
    try:
        COLUMNS_TO_CHECK_CORRELATION.remove(col)
    except ValueError:
        pass  # do nothing!
    
print(COLUMNS_TO_CHECK_CORRELATION)

['property_type', 'room_type', 'accommodates', 'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city', 'first_review', 'host_has_profile_pic', 'host_identity_verified', 'host_response_rate', 'host_since', 'instant_bookable', 'last_review', 'latitude', 'longitude', 'neighbourhood', 'number_of_reviews', 'review_scores_rating', 'bedrooms', 'beds']


In [108]:
def process_date(df, column):
    current_date = pd.to_datetime("today")
    df[column] = pd.to_datetime(df[column], errors='coerce')  # Coerce errors will turn problematic parsing into NaT
    df[column + '_days_since'] = (current_date - df[column]).dt.days
    df[column] = df[column + '_days_since']
    
def process_numerical(df, column):
    if column == 'host_response_rate':
        df[column] = df[column].str.replace('%', '')
    df[column] = df[column].astype(float)
    
def process_categorical(df, column):
    return pd.get_dummies(df[column], prefix=column)
    
def encode_df(X_columns, y_column):
    df = original_df.copy()
    all_columns = X_columns.copy()
    all_columns.append(y_column)

    df = df[all_columns].dropna(how='any')
    y = df[[y_column]]
    
    processed_columns = []
    for column in X_columns:
        if column in CATEGORICAL_COLUMNS:
            processed_columns.append(process_categorical(df, column))
        elif column in DATE_COLUMNS:
            process_date(df, column)
            processed_columns.append(df[column])
        elif column in NUMERICAL_COLUMNS:
            process_numerical(df, column)
            processed_columns.append(df[column])
            
    if y_column == "host_has_profile_pic":
        y = df[y_column].replace({'t': 1, 'f': 0})
    elif y_column == "host_identity_verified":
        y = df[y_column].replace({'t': 1, 'f': 0})
    elif y_column in DATE_COLUMNS:
        process_date(df, y_column)
        y = df[y_column]

    processed_columns = pd.concat(processed_columns, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(processed_columns, y, test_size=0.2)
    
    return X_train, X_test, y_train, y_test.to_numpy().squeeze()
    
    
    

In [73]:
def is_MCAR(column_name, correlation_threshold=0.7, chi2p_threshold=0.05):
    
    interesting_columns = []
    
    df = original_df.copy()

    for column in COLUMNS_TO_CHECK_CORRELATION:
        if column == column_name:
            continue
        
        # preprocess date
        
        if column in NUMERICAL_COLUMNS:
            
            process_numerical(df, column)
            correlation = df[column].corr(df[column_name].isnull(), method='pearson')

            # print("Correlation between '{}' and '{}':".format(column, column_name))
            # print(correlation)
            
            if abs(correlation) > correlation_threshold:
                interesting_columns.append(column)
        
        if column in DATE_COLUMNS:
            process_date(df, column)
            
            correlation = df[column].corr(df[column_name].isnull(), method='pearson')
            
            # print("Correlation between '{}' and '{}':".format(column, column_name))
            # print(correlation)
            
            if abs(correlation) > correlation_threshold:
                interesting_columns.append(column)
            

        if column in CATEGORICAL_COLUMNS:
            contingency_table = pd.crosstab(index=df[column], columns=df[column_name].isnull())
            chi2, p, _, _ = chi2_contingency(contingency_table)
            
            # print("Contingency correlation between '{}' and '{}':".format(column, column_name))
            # print(p)
            
            if abs(p) > chi2p_threshold:
                interesting_columns.append(column)
                
                
    print(f"columns that are correlated with {column_name} as a missing column:", interesting_columns)


In [74]:
# numerical = date, categorical

# numerical, numerical -> corr
# categorical, categorical -> chi2
# numerical, categorical -> foneway (anova)

def is_numerical(column_name):
    return column_name in NUMERICAL_COLUMNS or column_name in DATE_COLUMNS

def is_MAR(column_name, correlation_threshold=0.7, chi2p_threshold=0.05, anovap_threshold=0.05):
    
    interesting_columns = []
    
    df = original_df.copy()
    
    if column_name in DATE_COLUMNS:  
        process_date(df, column_name)
    elif column_name in NUMERICAL_COLUMNS:
        process_numerical(df, column_name)
    

    for column in COLUMNS_TO_CHECK_CORRELATION:
        if column == column_name:
            continue
        
        if column in DATE_COLUMNS:  
            process_date(df, column)
        elif column in NUMERICAL_COLUMNS:
            process_numerical(df, column)
            
        two_column_df = df[[column, column_name]].dropna()
        
        if is_numerical(column) and is_numerical(column_name):
            
            correlation = two_column_df[column].corr(two_column_df[column_name], method='pearson')

            # print("Correlation between '{}' and '{}':".format(column, column_name))
            # print(correlation)
            
            if abs(correlation) > correlation_threshold:
                interesting_columns.append(column)
                
        elif column in CATEGORICAL_COLUMNS and column_name in CATEGORICAL_COLUMNS:
            contingency_table = pd.crosstab(index=two_column_df[column], columns=two_column_df[column_name])
            chi2, p, _, _ = chi2_contingency(contingency_table)
            
            # print("Contingency correlation between '{}' and '{}':".format(column, column_name))
            # print(contingency_table)
            # print(p)
            
            if abs(p) > chi2p_threshold:
                interesting_columns.append(column)
            
        elif (is_numerical(column) and column_name in CATEGORICAL_COLUMNS) or (is_numerical(column_name) and column in CATEGORICAL_COLUMNS):
            
            numerical_column = column if is_numerical(column) else column_name
            categorical_column = column if (column in CATEGORICAL_COLUMNS) else column_name
            
            # le = LabelEncoder()
            # two_column_df[categorical_column] = le.fit_transform(two_column_df[categorical_column])
            
            # corr, p_value = stats.pointbiserialr(two_column_df[categorical_column], two_column_df[numerical_column])
                        
            groups = two_column_df.groupby(categorical_column)

            # Step 3: Perform ANOVA
            f_statistic, p_value = f_oneway(*[group[numerical_column] for _, group in groups])
            
            if abs(p_value) < anovap_threshold:
                interesting_columns.append(column)
        
                
    print(f"columns that are correlated with {column_name}", interesting_columns)


In [89]:
def train_numerical_y(X_train, X_test, y_train, y_test, num_clusters, k):
    # linear reg
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print('using linear reg', mean_squared_error(y_pred, y_test)) # MSE for numerical
    
    
    model = LogisticRegression()
    model.fit(X_train, (y_train * k).astype('int'))
    y_pred = model.predict(X_test) / k

    print('using logistic reg', mean_squared_error(y_pred, y_test)) # MSE for numerical
    

    # KNN
    model = KNeighborsClassifier(n_neighbors=num_clusters)
    
    # model.fit(X_train, y_train).astype('int')
    # y_pred = model.predict(X_test)
    
    model.fit(X_train, (y_train * k).astype('int'))
    y_pred = model.predict(X_test) / k

    print('using knn', mean_squared_error(y_pred, y_test))


    # randomforest
    model = RandomForestClassifier()
    
    # model.fit(X_train, y_train).astype('int')
    # y_pred = model.predict(X_test)
    
    model.fit(X_train, (y_train * k).astype('int'))
    y_pred = model.predict(X_test) / k

    print('using random forest', mean_squared_error(y_pred, y_test))


    # mean
    mean_value = y_train.mean()
    print('using mean', mean_squared_error(np.full(len(y_test), mean_value), y_test))

    # median
    mean_value = y_train.median()
    print('using median', mean_squared_error(np.full(len(y_test), mean_value), y_test))

    # mode
    mode_value = y_train.mode()
    print('using mode', mean_squared_error(np.full(len(y_test), mode_value), y_test))
    
    
def train_mcar(column_name):
    
    df = original_df.copy()

    if column_name in DATE_COLUMNS:  
        process_date(df, column_name)
    elif column_name in NUMERICAL_COLUMNS:
        process_numerical(df, column_name)
        
    y = df[column_name].copy().dropna().to_numpy().squeeze()
    
    y_train = y[:int(len(y) * 0.8)]
    y_test = y[int(len(y) * 0.8):]
    
    # mean
    mean_value = y_train.mean()
    print('using mean', mean_squared_error(np.full(len(y_test), mean_value), y_test))

    # median
    mean_value = np.median(y_train)
    print('using median', mean_squared_error(np.full(len(y_test), mean_value), y_test))

    # mode
    mode_value = mode(y_train)[0]
    print('using mode', mean_squared_error(np.full(len(y_test), mode_value), y_test))
    



### Per column analysis - bathrooms/bedrooms/beds

In [76]:
# Per column analysis - bathrooms

# by domain knowledge, it should be related to 'property_type', 'room_type', 'accommodates', 'bedrooms'

is_MCAR("bathrooms") # has some correlated columns, meaning it is not MCAR, it is either MAR or MNAR

is_MAR("bathrooms", correlation_threshold=0.7, chi2p_threshold=0.05, anovap_threshold=1e-100) # is_corr

X_columns = ['property_type', 'room_type']
y_column = "bathrooms"
X_train, X_test, y_train, y_test = encode_df(X_columns, y_column)
train_numerical_y(X_train, X_test, y_train, y_test, len(original_df[y_column].unique()), 2) # Linear regression has the lowest MSE


columns that are correlated with bathrooms as a missing column: ['bed_type', 'host_has_profile_pic', 'host_identity_verified']
columns that are correlated with bathrooms ['property_type', 'room_type', 'cancellation_policy']
using linear reg 0.3038899240722871


/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was pas

using logistic reg 0.3986166542650342
using knn 0.3986166542650342


/var/folders/l1/x0z_l9z95f984fpzdls50gmw0000gn/T/ipykernel_4984/817887534.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, (y_train * 2).astype('int'))


using random forest 0.3953189474396266
using mean 0.3445156785019402
using median 0.4006460123114388
using mode 0.4006460123114388


In [77]:
# Per column analysis - bedrooms

is_MCAR("bedrooms")

is_MAR("bedrooms")

X_columns = ['property_type', 'room_type', 'accommodates', 'bed_type', 'beds']
y_column = "bedrooms"
X_train, X_test, y_train, y_test = encode_df(X_columns, y_column)
train_numerical_y(X_train, X_test, y_train, y_test, len(original_df[y_column].unique()), 2) # Linear regression has the lowest MSE

columns that are correlated with bedrooms as a missing column: ['property_type', 'bed_type', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
columns that are correlated with bedrooms ['property_type', 'room_type', 'accommodates', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'host_identity_verified', 'beds']
using linear reg 0.3007885274376603


/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was pas

using logistic reg 0.32257846320346323
using knn 0.339353354978355


/var/folders/l1/x0z_l9z95f984fpzdls50gmw0000gn/T/ipykernel_4984/817887534.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, (y_train * 2).astype('int'))


using random forest 0.3040449134199134
using mean 0.7233747116042945
using median 0.7928841991341992
using mode 0.7928841991341992


In [78]:
# Per column analysis - beds

is_MCAR("beds")

is_MAR("beds")

X_columns = ['property_type', 'room_type', 'accommodates', 'bed_type', 'bedrooms']
y_column = "beds"
X_train, X_test, y_train, y_test = encode_df(X_columns, y_column)
train_numerical_y(X_train, X_test, y_train, y_test, len(original_df[y_column].unique()), 2) # linear regression has the lowest MSE

columns that are correlated with beds as a missing column: ['property_type', 'bed_type', 'host_has_profile_pic', 'instant_bookable']
columns that are correlated with beds ['property_type', 'room_type', 'accommodates', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'host_identity_verified', 'instant_bookable', 'bedrooms']
using linear reg 3.8017572631883366e+17


/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was pas

using logistic reg 0.5586444805194806
using knn 0.48856872294372294


/var/folders/l1/x0z_l9z95f984fpzdls50gmw0000gn/T/ipykernel_4984/817887534.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, (y_train * 2).astype('int'))


using random forest 0.4933712121212121
using mean 1.488282387821623
using median 1.977813852813853
using mode 1.977813852813853


In [79]:
# Per column analysis - first_review

is_MCAR("first_review") # MCAR

train_mcar("first_review") # use mean


columns that are correlated with first_review as a missing column: []
using mean 245738.65493215437
using median 263282.69442060083
using mode 370669.9900429185


In [80]:
# Per column analysis - last_review

is_MCAR("last_review")  # MCAR

train_mcar("last_review")  # use mean


columns that are correlated with last_review as a missing column: []
using mean 53170.763559042054
using median 54947.973749678305
using mode 55120.88650596208


In [90]:
# Per column analysis - review_scores_rating

is_MCAR("review_scores_rating")

is_MAR("review_scores_rating")

X_columns = ['property_type', 'room_type', 'cleaning_fee', 'instant_bookable']
y_column = "review_scores_rating"
X_train, X_test, y_train, y_test = encode_df(X_columns, y_column)
train_numerical_y(X_train, X_test, y_train, y_test, 10, 100) # linear regression has the lowest MSE

columns that are correlated with review_scores_rating as a missing column: []
columns that are correlated with review_scores_rating ['property_type', 'room_type', 'cancellation_policy', 'cleaning_fee', 'host_identity_verified', 'instant_bookable']
using linear reg 57.82099802383128


/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was pas

using logistic reg 93.05985363303711
using knn 79.69245513155602


/var/folders/l1/x0z_l9z95f984fpzdls50gmw0000gn/T/ipykernel_4984/3344523772.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, (y_train * k).astype('int'))


using random forest 92.8031015856421
using mean 58.60268201741361
using median 62.094528663530234
using mode 93.05671719811814


In [103]:
# Per column analysis - host_has_profile_pic

is_MCAR("host_has_profile_pic")

is_MAR("host_has_profile_pic")

X_columns = ['property_type', 'room_type', 'cleaning_fee', 'instant_bookable']
y_column = "host_has_profile_pic"
X_train, X_test, y_train, y_test = encode_df(X_columns, y_column)
train_numerical_y(X_train, X_test, y_train, y_test, 2, 1) # abt the same but linear regression has the lowest MSE

columns that are correlated with host_has_profile_pic as a missing column: ['bed_type', 'cancellation_policy', 'cleaning_fee', 'host_identity_verified', 'instant_bookable']
columns that are correlated with host_has_profile_pic ['property_type', 'room_type', 'bed_type', 'first_review', 'host_response_rate', 'host_since', 'number_of_reviews']
using linear reg 229169436281819.84
using logistic reg 0.0031112614135948595
using knn 0.02238755495434562
using random forest 0.0031112614135948595
using mean 0.0031015860267421726
using median 0.0031112614135948595
using mode 0.0031112614135948595


In [106]:
# Per column analysis - host_identity_verified

is_MCAR("host_identity_verified")

is_MAR("host_identity_verified")

X_columns = ['accommodates', 'bathrooms', 'first_review', 'host_response_rate', 'host_since', 'number_of_reviews', 'review_scores_rating', 'bedrooms', 'beds']
y_column = "host_identity_verified"
X_train, X_test, y_train, y_test = encode_df(X_columns, y_column)
train_numerical_y(X_train, X_test, y_train, y_test, 2, 1) # linear regression has the lowest MSE

columns that are correlated with host_identity_verified as a missing column: ['bed_type', 'cancellation_policy', 'cleaning_fee', 'host_has_profile_pic', 'instant_bookable']
columns that are correlated with host_identity_verified ['accommodates', 'bathrooms', 'first_review', 'host_response_rate', 'host_since', 'last_review', 'number_of_reviews', 'review_scores_rating', 'bedrooms', 'beds']
using linear reg 0.17408303667099453


/Users/chrystalquek/opt/anaconda3/envs/aml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


using logistic reg 0.23969449675664364
using knn 0.36702238962125966
using random forest 0.22421008579200669
using mean 0.19394606005739892
using median 0.2631303620004185
using mode 0.2631303620004185


In [91]:
# Per column analysis - host_response_rate

is_MCAR("host_response_rate")

train_mcar("host_response_rate")  # use mean

columns that are correlated with host_response_rate as a missing column: []
columns that are correlated with host_response_rate ['property_type', 'room_type', 'cancellation_policy', 'cleaning_fee', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
using mean 253.9039285575161
using median 283.5200214995969
using mode 283.5200214995969


In [109]:
# Per column analysis - host_since

is_MCAR("host_since")
is_MAR("host_since")


X_columns = ['property_type', 'room_type', 'bed_type', 'cancellation_policy', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
y_column = "host_since"
X_train, X_test, y_train, y_test = encode_df(X_columns, y_column)
train_numerical_y(X_train, X_test, y_train, y_test, 10, 1) # linear regression has the lowest MSE

columns that are correlated with host_since as a missing column: ['bed_type', 'cancellation_policy', 'cleaning_fee', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
columns that are correlated with host_since ['property_type', 'room_type', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
using linear reg 371163.9804024349


KeyboardInterrupt: 

In [ ]:
# Per column analysis - neighbourhood, zipcode


print(original_df['neighbourhood'].unique())